# Sampler dev notebook for Metallurgy project

### Notes, thoughts, and ideas
Collected here are some notes to remember, thoughts on the code, and ideas for future implementation.

### Features to implement
- Create midi file
    - This can be an additional output option for automatic transcription
    - Use mido library
    - Translate note information to MIDI
        - For each note data provided by the analysis, create an associated MIDI note
- ADSR for the samples
    - Initially implement attack and release
    - Attack: Time in ms that the sample ramps in from 0 to 1.
        - Min = 1 ms? This would be 44 samples at 44.1k. 0.5 ms would be 22 samples. That's a plenty short period of time.
        - Max = 1 sec?
    - Release: Time in ms the sample ramps out from 1 to 0.
    - Should the scaling be linear? Exponential?
- Move the `add_sample()` function to the functions cell for general use.
- The `add_sample()` function may need adjustment when Keon let's me know what the formatting will be for data array.

In [1]:
# imports
import os
import IPython
import numpy as np
from scipy.io import wavfile
import wavio
import librosa

In [19]:
# functions

def load_wav(file):
    '''
    Import 24-bit wav files and convert to 32-bit float
    
    Parameters
    ----------
    file       : wav file
        Wav file to be loaded.
    
    Returns
    -------
    data          : np.array [shape=(?), dtype=np.float32]
        Wav sample data
    rate          : int
        Samplerate associated with the wav file loaded
    
    Notes
    -----
    Conversion    : May need to add conversion cases for 16-bit audio. 32-bit
                    wav files are uncommon so probably not necessary for this
                    project.
    '''
    wav = wavio.read(file)
    # convert from 24-bit in to 32-bit float
    data = np.zeros(len(wav.data), dtype=np.float32)
    if wav.data.dtype==np.int32 and wav.sampwidth==3:
        for smp in range(len(wav.data)):
            data[smp] = wav.data[smp] / 8388608.0
    return data, wav.rate



def convert_samplerate(data, original_sr, target_sr=44100):
    '''
    Convert audio of an aribtrary samplerate to 44.1 kHz
    Input:      wav data, wave data samplerate
    Output:     resampled wav data
    '''
    if original_sr != target_sr:
        resampled_data = librosa.resample(data, original_sr, target_sr)
        return resampled_data
    else:
        return data
    

    
def hz_to_midi(frequency):
    '''
    Convert a frequency to its corresponding midi value.
    
    Parameters
    ----------
    frequency       : float
        frequency to convert
    
    Returns
    -------
    midi_note_num   : float
        MIDI note number (fractional) that corresponds to the input frequency
    '''
    return 12 * (np.log2(frequency) - np.log2(440.0)) + 69



def add_sample(input_array, output_audio):
    '''
    Add an audio sample to the audio output.
    
    Parameters
    ----------
    input_array     : np.ndarray [shape=(2,), dtype=float]
        Input array to be processed
        Element 0   : Note frequency in hz
        Element 1   : Time placement in output file in sec
        
    output_audio    : np.ndarray [shape=(n,), dtype=np.float32]
        Array containing audio data for final output
        
    Returns
    -------
    output_audio is returned after adding the audio sample
    '''
    # Calculate midi note of input frequency (currently as an int)
    midi_note_num = np.rint(hz_to_midi(input_array[0])).astype(int)
    
    # Determine the file name of the sample to use
    # TODO: convert this to own function?
    sample_file_name = str(midi_note_num) + "/Lead-" + str(midi_note_num) + "-6.wav"
    
    # Determine the location of the sample to use
    sample_path = os.path.join("../Audio/MetallurgySamples/", sample_file_name)
    
    # Convert time in sec to sample position
    start_pos = int(_samplerate * input_array[1])
    
    # Load sample data
    sample, sr = load_wav(sample_path)
    
    # Check sample samplerate
    if sr != 44100:
        sample = convert_samplerate(sample, sr)
    
    # Check output_audio array length to make sure sample being added doesn't overrun it
    sample_length = len(sample)
    space_left = len(output_audio) - start_pos
    if sample_length >= space_left:
        padding = np.zeros(sample_length - space_left + 1, dtype=np.float32)
        output_audio = np.concatenate([output_audio, padding])
    
    # Ramp in
    sample = add_attack(sample)
        
    # Ramp out
    sample = add_release(sample)
    
    # Add sample data to output_audio array
    for smp in range(len(sample)):
        output_audio[start_pos + smp] += sample[smp]
    
    return output_audio



def add_attack(sample, attack_time=1, samplerate=44100):
    '''
    Add an attack ramp to a sample.
    
    Parameters
    ----------
    sample : np.ndarry [ shape = (n,), dtype=np.float32 ]
        Sample to add attack ramp to
        
    attack_time : float
        Ramp length in milliseconds. Must be >= 0 and less than the
        length of the input sample.
        
    samplerate : int > 0
        The samplerate for the sample.
        
    Returns
    -------
    processed_sample : np.ndarray [ shape = (n,), dtype=np.float32 ]
        Returns the processed sample array.
    '''
    
    assert attack_time >= 0, "Attack time is negative."
    assert attack_time < len(sample), "Attack time is longer than sample."
    assert samplerate > 0, "Samplerate is not a positive integer."
    
    ramp_time = int(np.floor(attack_time * samplerate / 1000))
    for smp in range(ramp_time):
        sample[smp] *= smp / ramp_time
    return sample



def add_release(sample, release_time=1, samplerate=44100):
    '''
    Add a release ramp to a sample.
    
    Parameters
    ----------
    sample : np.ndarray [ shape = (n,), dtype=np.float32 ]
        Sample to add release ramp to
        
    release_time : float
        Rampl length in milliseconds. Must be >= 0 and less than the
        length of the input sample.
        
    samplerate : int > 0
        The samplerate for the sample.
        
    Returns
    -------
    processed_sample : np.ndarray [ shape = (n,), dtype=np.float32 ]
        Returns the processed sample array.
    
    '''
    
    assert release_time >= 0, "Release time is negative."
    assert release_time < len(sample), "Release time is longer than the sample."
    assert samplerate > 0, "Samplerate is not a positive integer."
    
    ramp_time = int(np.floor(release_time * samplerate / 1000))
    for smp in range(ramp_time):
        sample[-smp-1] *= smp/ramp_time
    
    return sample

#TODO: Create combined AR (attack/release function) that calls add_attack, add_release, and checks 

## Setup

We'll process all audio as 32-bit floats in the range [-1.0,1.0] and convert to 24-bit int values at the end. 32-bit float allows audio from different source rates to interact naturally without the need for constant conversion. Just convert at the beginning and end.

We should use just one samplerate for everything. I prefer 48 kHz to 44.1 kHz, but 44.1 kHz seems to be the standard due to the influence of CD audio. We should create a function to the check the samplerate of audio as it is loaded and if it not 44.1, then convert it.

In [20]:
_samplerate = 44100

In [21]:
# create an audio array for adding samples to
empty_output = np.zeros(1, dtype = np.float32)
sample_audio_mono = np.zeros(10*_samplerate, dtype=np.float32)
sample_audio_stereo = np.zeros((10*_samplerate,2), dtype=np.float32)

In [22]:
sample_audio_mono.shape

(441000,)

In [23]:
sample_file = "../Audio/MetallurgySamples/72/Lead-72-6.wav"

In [24]:
sample, sr = load_wav(sample_file)

In [25]:
# Resample if necessary
sample = convert_samplerate(sample, sr)

In [26]:
for smp in range(len(sample)):
    sample_audio_mono[smp] += sample[smp]

In [29]:
test_note = np.array([440.0, 8.0])

In [30]:
hz_to_midi(test_note[0])

69.0

In [31]:
sample_audio_mono = add_sample(test_note, sample_audio_mono)

In [32]:
print(sample_audio_mono.shape)

(509246,)


In [35]:
# output the sampler audio to a wav file
wavfile.write("../Audio/test/sampler_output_mono.wav", _samplerate, sample_audio_mono)

In [36]:
# create IPython playback
IPython.display.Audio('../Audio/test/sampler_output_mono.wav')